# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Benisty  ->  M. Benisty  |  ['M. Benisty']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
J. Liu  ->  J. Liu  |  ['J. Liu']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Liu  ->  J. Liu  |  ['J. Liu']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Liu  ->  J. Liu  |  ['J. Liu']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


Y. Wang  ->  Y. Wang  |  ['Y. Wang']
J. Liu  ->  J. Liu  |  ['J. Liu']
Arxiv has 91 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2504.03833


extracting tarball to tmp_2504.03833...

 done.


M. Benisty  ->  M. Benisty  |  ['M. Benisty']


...

 done.
Retrieving document from  https://arxiv.org/e-print/2504.04648


extracting tarball to tmp_2504.04648...

 done.
Retrieving document from  https://arxiv.org/e-print/2504.05233


extracting tarball to tmp_2504.05233...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.03833-b31b1b.svg)](https://arxiv.org/abs/2504.03833) | **Radial variations in nitrogen, carbon, and hydrogen fractionation in the PDS 70 planet-hosting disk**  |
|| L. Rampinelli, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2025-04-08*|
|*Comments*| *18 pages, 10 figures, 3 tables*|
|**Abstract**|            Element isotopic ratios are powerful tools to reconstruct the journey of planetary material, from the parental molecular cloud to protoplanetary disks, where planets form and accrete their atmosphere. Radial variations in isotopic ratios in protoplanetary disks reveal local pathways which can critically affect the degree of isotope fractionation of planetary material. In this work we present spatially-resolved profiles of the 14N/15N, 12C/13C, and D/H isotopic ratios of the HCN molecule in the PDS 70 disk, which hosts two actively-accreting giant planets. ALMA high spatial resolution observations of HCN, H13CN, HC15N, and DCN reveal radial variations of fractionation profiles. We extract the HCN/HC15N ratio out to ~120 au, which shows a decreasing trend outside the inner cavity wall of the PDS 70 disk located at ~50 au. We suggest that the radial variations observed in the HCN/HC15N ratio are linked to isotope selective photodissociation of N2. We leverage the spectrally resolved hyperfine component of the HCN line to extract the radial profile of the HCN/H13CN ratio between ~40 and 90 au, obtaining a value consistent with the ISM 12C/13C ratio. The deuteration profile is also mostly constant throughout the disk extent, with a DCN/HCN ratio ~0.02, in line with other disk-averaged values and radial profiles in disks around T Tauri stars. The extracted radial profiles of isotopologue ratios show how different fractionation processes dominate at different spatial scales in the planet-hosting disk of PDS 70.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.03836-b31b1b.svg)](https://arxiv.org/abs/2504.03836) | **Quantifying the Impact of Incompleteness on Identifying and Interpreting Galaxy Protocluster Populations with the TNG-Cluster Simulation**  |
|| D. C. Baxter, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2025-04-08*|
|*Comments*| *Key Figures: 3, 7, 9, and 13. The manuscript contains 28 pages and 14 figures. Submitted to ApJ. Comments are welcome*|
|**Abstract**|            We use the TNG-Cluster simulation to examine how stellar mass and star formation rate (SFR) incompleteness affect the identification of density peaks within galaxy protoclusters at different redshifts. We focus on 352 protoclusters, defined as the set of galaxies that will reside within the virialized region of a $z=0$ cluster halo with total mass $\sim10^{14.3-15.5}~{\rm M}_{\odot}$, and consider only galaxies with ${M}_{\star}> 10^{8.5}~{\rm M}_{\odot}$ (our baseline) at any redshift. We find that ${M}_{\star}$-limited (${M}_{\star} > 10^{9.5}~{\rm M}_{\odot}$) and SFR-limited ($\rm{SFR} > 10~{\rm M}_{\odot} \mathrm{yr}^{-1}$) subpopulations only recover the true highest galaxy density peak in $\lesssim40\%$ of cases within an accuracy of $1.0$ pMpc ($\sim 2-2.5$ arcmin) at $z > 2$. We find that at $z>2$ the highest galaxy density peaks do not generally coincide with the highest dark matter or stellar mass density peaks, and this separation is larger than 0.5 pMpc in $\sim80\%$ of cases. Consequently, the region surrounding the true highest galaxy density peaks are not generally sites of enhanced star formation or accelerated mass growth relative to the remainder of the protocluster. Lastly, we find that the $\sim4$ arcmin apertures typically used to define spectroscopically-confirmed protoclusters are generally much smaller than the $8$ arcmin apertures needed to study the progenitors of the most massive galaxy clusters at $z > 2$.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.03856-b31b1b.svg)](https://arxiv.org/abs/2504.03856) | **Optical and Near-infrared Observations of SN 2023ixf for over 600 days after the Explosion**  |
|| G. Li, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-04-08*|
|*Comments*| **|
|**Abstract**|  this http URL present a comprehensive photometric and spectroscopic study of the nearby Type II supernova (SN) 2023ixf, with our extensive observations spanning the phases from ~3 to over 600 days after the first light.\\ this http URL aim of this study is to obtain key information on the explosion properties of SN\,2023ixf and the nature of its progenitor.\\ this http URL observational properties of SN\,2023ixf are compared with those of a sample of Type IIP/L SNe to investigate commonalities and diversities. We conduct a detailed analysis of temporal evolution of major spectral features observed throughout different phases of the SN\,2023ixf explosion. Several interpretations are addressed through a comparison between the data and the model spectra for progenitor stars within a range of zero-age main sequence (ZAMS) masses.\\ this http URL observations indicate that SN\,2023ixf is a transitional SN that bridges the gap between Type IIP and IIL subclasses of H-rich SNe, characterized by a relatively short plateau ($\lesssim 70$\,d) in the light curve. It shows a rather prompt spectroscopic evolution toward the nebular phase; emission lines of Na, O, H, and Ca in nebular spectra all exhibit multipeak profiles, which might be attributed to bipolar distribution of the ejecta. In particular, the H$\alpha$ profile can be separated into two central peaked components (with a velocity of about 1500\,km\,s$^{-1}$) that is likely due to nickel-powered ejecta and two outer peak/box components (with a velocity extending up to ~8000 km\,s$^{-1}$) that can arise from interaction of the outermost ejecta with a circumstellar shell at a distance of $\sim6.2\times10^{15}$cm. The nebular-phase spectra of SN\,2023ixf show good agreement with those predicted by model spectra for progenitor stars with a ZAMS mass ranging from 15 to 19\,M${_\odot}$. A distance $D = 6.35^{+0.31}_{-0.39}$\,Mpc is estimated for M101.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.04387-b31b1b.svg)](https://arxiv.org/abs/2504.04387) | **A Multi-Mechanism Hybrid Model of Peaked-Spectrum Radio Sources**  |
|| G.-C. Sun, et al. -- incl., <mark>Y. Wang</mark>, <mark>X. Zhang</mark> |
|*Appeared on*| *2025-04-08*|
|*Comments*| *23 pages, 9 figures, 5 tables. Accepted for publication in ApJ*|
|**Abstract**|            The peaked-spectrum (PS) sources exhibit turnover characteristics in their broad radio spectra. However, the mechanism underlying this phenomenon remains elusive. The two most common hypotheses are synchrotron self-absorption (SSA) and free-free absorption (FFA). By incorporating multiple absorption scenarios, we propose a multi-mechanism hybrid (MMH) model, which aligns well with current observational data and provides a good physical explanation. Using the GLEAM survey data, we identified a sample of 4,315 sources with peak frequencies approximately between 72--3000 MHz, most of which are MHz-peaked-spectrum sources (MPS). Our analysis shows that instead of SSA, the FFA is the dominant mechanism in producing the spectral turnover for most of the sources in this sample. The index of the optically thick spectrum alpha_thick has a lower boundary due to the FFA, and the steeper alpha_thick indicates a complex multi-absorption mechanism. In particular, the external FFA produces substantial alpha_thick, which exhibits a weak correlation with the peak frequency. Future ultra-long wavelength observations would also provide data on the spectrum of these sources at even lower frequencies. Determining the absorption mechanism that shaped the spectrum of these sources would be a crucial part of understanding their nature.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.04393-b31b1b.svg)](https://arxiv.org/abs/2504.04393) | **Supernovae at Distances < 40 Mpc: I.Catalogues and fractions of Supernovae in a Complete Sample**  |
|| X. Ma, et al. -- incl., <mark>J. Liu</mark>, <mark>X. Zhang</mark> |
|*Appeared on*| *2025-04-08*|
|*Comments*| *22 pages, 27 figures, to be published in A&A*|
|**Abstract**|  this http URL is the first paper of a series aiming to determine the fractions and birth rates of various types of supernovae (SNe) in the local Universe. Aims. In this paper, we aim to construct a complete sample of SNe in the nearby universe and provide more precise measurement of subtype fractions. this http URL carefully selected our SN sample at a distance of < 40 Mpc mainly from wide-field surveys conducted over the years from 2016 to 2023. this http URL sample contains a total of 211 SNe, including 109 SNe II, 69 SNe Ia, and 33 SNe Ibc. With the aid of sufficient spectra, we can obtain relatively accurate subtype classifications for all SNe in this sample. After corrections for the Malmquist bias, this volume-limited sample gives fractions of SNe Ia, SNe Ibc, and SNe II as $30.4^{+3.7}_{-11.5}\%$, $16.3^{+3.7}_{-7.4}\%$, and $53.3^{+9.5}_{-18.7}\%$, this http URL the SN Ia sample, the fraction of the 91T-like subtype becomes relatively low (~5.4\%), while that of the 02cx-like subtype shows a moderate increase (~6.8\%). In the SN Ibc sample, we find significant fractions of broadlined SNe Ic (~18.0\%) and SNe Ibn (~8.8\%). The fraction of 87A-like subtype is determined as ~2.3\% for the first time, indicating rare explosions from blue supergiant stars. We find that SNe Ia show a double peak number distribution in S0- and Sc-type host galaxies, which may serve as a straightforward evidence for the presence of "prompt" and "delayed" progenitor components giving rise to SN Ia explosions. Several subtypes of SNe such as 02cx-like SNe Ia, broadlined SNe Ic, SNe IIn (and perhaps SNe Ibn) are found to occur preferentially in less massive spiral galaxies, favoring their associations with young stellar progenitors. Moreover, the 02cx-like subtype shows a trend of exploding in the outer skirt of their hosts, suggestive of metal-poor progenitors.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.04507-b31b1b.svg)](https://arxiv.org/abs/2504.04507) | **Supernova Sample at Distances < 40 Mpc: II. Supernova Rate in the Local Universe**  |
|| X. Ma, et al. -- incl., <mark>J. Liu</mark>, <mark>X. Zhang</mark> |
|*Appeared on*| *2025-04-08*|
|*Comments*| *14 pages, 11 figures, to be published in A&A*|
|**Abstract**|  this http URL is the second paper of a series aiming to determine the birth rates of supernovae in the local Universe. Aims. In this paper, we aim to estimate the SN rates in the local universe and fit the delay-time distribution of SNe Ia to put constraints on their progenitor scenarios. this http URL performed a Monte-Carlo simulation to estimate the volumetric rates with the nearby SN sample introduced in Paper I of the series. The rate evolution of core-collapse SNe well traces the evolution of cosmic star formation history; while that of SNe Ia involves the convolution of cosmic star-formation history and a two-component delay-time distribution including a power law and a Gaussian component. this http URL volumetric rates of type Ia, Ibc and II SNe are derived as $0.325\pm0.040^{+0.016}_{-0.010}$, $0.160\pm0.028^{+0.044}_{-0.014}$, and $0.528\pm0.051^{+0.162}_{-0.013}$ (in unit of $10^{-4} yr^{-1} Mpc^{-3} h^3_{70}$), respectively. The rate of CCSNe is consistent with previous estimates. The newly derived local SN Ia rate is larger than existing results given at redshifts 0.01 < z < 0.1, favoring an increased rate from the universe at z ~ 0.1 to the local universe. A two-component model can well fit the rate variation, with the power law component accounting for the rate evolution at larger redshifts and the Gaussian component with a delay time of 12.63$\pm$0.38 Gyr accounting for the local rate evolution. This delayed component with such a longer delay time suggests that the progenitors of these SNe Ia were formed at around 1 Gyr after the birth of the universe, which could only be explained by a double-degenerate progenitor scenario. This is evidenced by the comparison with the PTF sample of SNe Ia at z = 0.073, which reveals that the increase in SN Ia rate at z < 0.01 is primarily due to the SNe Ia of massive E and S0 galaxies with old stellar populations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.04648-b31b1b.svg)](https://arxiv.org/abs/2504.04648) | **Basic Pattern of Three-dimensional Magnetic Reconnection within Strongly Turbulent Current Sheets**  |
|| <mark>Y. Wang</mark>, X. Cheng, M. Ding |
|*Appeared on*| *2025-04-08*|
|*Comments*| *18 pages, 14 figure, accepted for publication in ApJ*|
|**Abstract**|            Magnetic reconnection is a fundamental mechanism of driving eruptive phenomena of different scales and may be coupled with turbulence as suggested by recent remote-sensing and in-situ observations. However, the specific physics behind the complex three-dimensional (3D) turbulent reconnection remains mysterious. Here, we develop a novel methodology to identify and analyze multitudes of multi-scale reconnection fragments within a strongly turbulent current sheet (CS) and apply it to a state-of-the-art numerical simulation of turbulent reconnection for solar flares. It is determined that the reconnection fragments tend to appear as quasi-2D sheets forming along local magnetic flux surfaces, and, due to strong turbulence, their reconnection flow velocities and reconnection rates are significantly broadened statistically but are scale-independent. Each reconnection fragment is found to be surrounded by strongly fluctuated in/out-flows and has a widely distributed reconnection rate, mainly in the range of 0.01-0.1. The results, for the first time, provide quantitative measurements of 3D magnetic reconnection in strongly turbulent flare CSs, offering insights into the cascading laws of 3D reconnection in other turbulent plasmas.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.05233-b31b1b.svg)](https://arxiv.org/abs/2504.05233) | **Formation of Near-surface Atmospheric Inversion and Surface Inversion in Hothouse Climates**  |
|| <mark>J. Liu</mark>, F. Ding, J. Yang |
|*Appeared on*| *2025-04-08*|
|*Comments*| *20 pages, 12 figures, accepted for publication in the Astrophysical Journal*|
|**Abstract**|            A hothouse climate may develop throughout Earth's history and its warming future and on potentially habitable exoplanets near the inner edge of the habitable zone. Previous studies suggested that near-surface atmospheric inversion (NAIV) with planetary boundary air temperature being higher than the air temperature adjacent to the surface, is a pronounced phenomenon in hothouse climates. However, the underlying mechanisms are unclear. Here we show that lower-tropospheric radiative heating is necessary but not independently sufficient in forming the NAIV. Instead, the dynamic heating induced by large-scale subsidence is essential. With the prescribed reasonable large-scale subsidence, NAIV appears in small-domain cloud-resolving simulations, which was not observed in previous studies. Surface evaporative cooling also contributes to the formation of the NAIV. Besides NAIV, we find that surface inversion (SIV) with the air adjacent to the surface being warmer than the underlying sea surface is also a distinct phenomenon in hothouse climates. SIV is caused by strong surface evaporative cooling and large atmospheric shortwave absorption. These two types of inversion strongly stabilize the atmosphere, weaken atmospheric circulation, dry the free troposphere, and suppress the hydrological cycle.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2504.03833/./images/fractionation_hyperfine_12C13C_N+H_tau_b7res_dTb.png', 'tmp_2504.03833/./images/rot_diagram_combined_tau_profiles_dTb.png', 'tmp_2504.03833/./images/frac_compare_disks_tau_dTb.png']
copying  tmp_2504.03833/./images/fractionation_hyperfine_12C13C_N+H_tau_b7res_dTb.png to _build/html/
copying  tmp_2504.03833/./images/rot_diagram_combined_tau_profiles_dTb.png to _build/html/
copying  tmp_2504.03833/./images/frac_compare_disks_tau_dTb.png to _build/html/
exported in  _build/html/2504.03833.md
    + _build/html/tmp_2504.03833/./images/fractionation_hyperfine_12C13C_N+H_tau_b7res_dTb.png
    + _build/html/tmp_2504.03833/./images/rot_diagram_combined_tau_profiles_dTb.png
    + _build/html/tmp_2504.03833/./images/frac_compare_disks_tau_dTb.png
found figures []
exported in  _build/html/2504.03836.md


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\as}{^{\prime\prime}}$
$\newcommand$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# Radial variations in nitrogen, carbon, and hydrogen fractionation in the PDS 70 planet-hosting disk

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.03833-b31b1b.svg)](https://arxiv.org/abs/2504.03833)<mark>Appeared on: 2025-04-08</mark> -  _18 pages, 10 figures, 3 tables_

</div>
<div id="authors">

L. Rampinelli, et al. -- incl., <mark>M. Benisty</mark>

</div>
<div id="abstract">

**Abstract:** Element isotopic ratios are powerful tools to reconstruct the journey of planetary material, from the parental molecular cloud to protoplanetary disks, where planets form and accrete their atmosphere. Radial variations in isotopic ratios in protoplanetary disks reveal local pathways which can critically affect the degree of isotope fractionation of planetary material. In this work we present spatially-resolved profiles of the $\ce{^14N}$ / $\ce{^15N}$ , $\ce{^12C}$ / $\ce{^13C}$ , and D/H isotopic ratios of the HCN molecule in the PDS 70 disk, which hosts two actively-accreting giant planets. ALMA high spatial resolution observations of HCN, $\ce{H^13CN}$ , $\ce{HC^15N}$ , and DCN reveal radial variations of fractionation profiles. We extract the HCN/ $\ce{HC^15N}$ ratio out to $\sim$ 120 au, which shows a decreasing trend outside the inner cavity wall of the PDS 70 disk located at $\sim50$ au. We suggest that the radial variations observed in the HCN/ $\ce{HC^15N}$ ratio are linked to isotope selective photodissociation of $\ce{N2}$ . We leverage the spectrally resolved hyperfine component of the HCN line to extract the radial profile of the HCN/ $\ce{H^13CN}$ ratio between $\sim$ 40 and 90 au, obtaining a value consistent with the ISM $\ce{^12C}$ / $\ce{^13C}$ ratio. The deuteration profile is also mostly constant throughout the disk extent, with a DCN/HCN ratio $\sim$ 0.02, in line with other disk-averaged values and radial profiles in disks around T Tauri stars. The extracted radial profiles of isotopologue ratios show how different fractionation processes dominate at different spatial scales in the planet-hosting disk of PDS 70.

</div>

<div id="div_fig1">

<img src="tmp_2504.03833/./images/fractionation_hyperfine_12C13C_N+H_tau_b7res_dTb.png" alt="Fig5" width="100%"/>

**Figure 5. -** Nitrogen, carbon, and hydrogen fractionation of the HCN molecule. Left panel: $69\times\ce{H^13CN}/\ce{HC^15N}$(dark blue) and \ce{HCN}/\ce{HC^15N}(light blue) profiles. Middle panel:  \ce{HCN}/\ce{H^13CN}. Right panel: $69\times\ce{H^13CN}/\ce{DCN}$(pink) and \ce{HCN}/\ce{DCN}(purple) profiles. Ribbons show the 16th and 84th percentiles of the posterior distributions of isotopologue ratios. The dashed horizontal black lines show the ISM values of $\ce{^14N}/\ce{^15N}=274$(blue) and of $\ce{^12C}/\ce{^13C}=69$, while the ISM values of D/H is not indicated, as it is $\sim10^{-5}$\citep[see references in][]{nomura2022isotopic}. The marker length in each legend shows the major axis of the beam of cubes used to retrieve the corresponding profiles in the plots. The black dotted line shows the $855\mu$m continuum integrated intensity profile  ([Isella, Benisty and Teague 2019](), [Benisty, Bae and Facchini 2021]()) . (*fig:12C13C*)

</div>
<div id="div_fig2">

<img src="tmp_2504.03833/./images/rot_diagram_combined_tau_profiles_dTb.png" alt="Fig3" width="100%"/>

**Figure 3. -** Top panels: integrated intensity profiles of lines of HCN isotopologues. The ribbons show the standard deviation across each annulus divided by the square root of the number of independent beams. The intensity is expressed in brightness temperature under the Rayleigh-Jeans approximation. The black dashed line is the integrated intensity profile of the 855 $\mu$m continuum  ([Isella, Benisty and Teague 2019](), [Benisty, Bae and Facchini 2021]()) . The hatched region shows the beam major axis of each line. Bottom row: results of the radial rotational diagram analysis. Solid lines in the left and middle panels show the 50th percentile of column density profiles (\ce{H^13CN} in blue and \ce{HC^15N} in pink), and of the excitation temperature (black). Ribbons show the 16th and 84th percentiles of the posterior distributions. The right panel represents the radial profiles of the optical depth for the analyzed molecular lines of \ce{H^13CN} and \ce{HC^15N}, and the related uncertainty shown for the \ce{H^13CN}(3-2) line, for reference, as it is representative of the typical uncertainty. The dashed black line shows the $\tau=1$ level. (*fig:profiles_rotdiagr*)

</div>
<div id="div_fig3">

<img src="tmp_2504.03833/./images/frac_compare_disks_tau_dTb.png" alt="Fig6" width="100%"/>

**Figure 6. -** Radial profiles of nitrogen, carbon, and hydrogen fractionation of HCN in disks. The red profiles in the three panels show the results for the PDS 70 disk (solid lines and ribbons show the 50th, 16th, and 84th percentiles of the posterior distributions, respectively), while the horizontal dashed lines indicate the ISM values  ([Nomura, Furuya and Cordiner 2023](), [Ritchey, Federman and Lambert 2015](), [ and Wilson 1999](), [Linsky, Draine and Moos 2006]()) . The isotopic ratios profiles for the PDS 70 disk are compared to the ones for the TW Hya  ([Hily-Blant, Kastner and Forveille 2019]())  and V4046 Sgr disks ( ([Nomura, Furuya and Cordiner 2023]()) , Guzmán et al. in prep.) for nitrogen in the left panel, the TW Hya disk  ([Hily-Blant, Kastner and Forveille 2019]())  for carbon in the middle panel, and to the disks in the MAPS survey  ([Cataldi, Yamato and Aikawa 2021]())  for hydrogen in the right panel. The nitrogen and hydrogen fractionation profiles are obtained by converting the \ce{H^13CN} into an HCN column density, assuming a fixed $\ce{^12C}/\ce{^13C}=69$ ratio. The grey line on top of each panel shows the major axis of the beam of the observations used to extract the profiles for the PDS 70 disk. (*fig:frac_disks*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.03833"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\pgfsyspdfmark}[3]$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\vcirc}{v_{\rm{circ}}}$
$\newcommand{\vmax}{{\it V}_{\rm{max}}}$
$\newcommand{\rmax}{R_{\rm{max}}}$
$\newcommand{\mtwo}{M_{\rm{200}}}$
$\newcommand{\rtwo}{R_{\rm{200c}}}$
$\newcommand{\rctwo}{R_{\rm{200c}}}$
$\newcommand{\mhalf}{M_{1/2}}$
$\newcommand{\rhalf}{R_{1/2}}$
$\newcommand{\mmax}{M_{\rm max}}$
$\newcommand{\msub}{M_{\rm{sub}}}$
$\newcommand{\mvir}{{\rm M}_{\rm{vir}}}$
$\newcommand{\mhalo}{{M}_{\rm{halo}}}$
$\newcommand{\mpeak}{{M}_{\rm{peak}}}$
$\newcommand{\rvir}{R_{\rm vir}}$
$\newcommand{\vvir}{V_{\rm{vir}}}$
$\newcommand{\vlos}{V_{\rm{los}}}$
$\newcommand{\vpeak}{{\it V}_{\rm{peak}}}$
$\newcommand{\dd}{{\rm d}}$
$\newcommand{\mstar}{{M}_{\star}}$
$\newcommand{\minfall}{{M}_{\star, \rm{infall}}}$
$\newcommand{\mhi}{{M}_{\rm H{\scriptsize I}}}$
$\newcommand{\lstar}{L^{*}}$
$\newcommand{\msun}{{\rm M}_{\odot}}$
$\newcommand{\lsun}{L_{\odot}}$
$\newcommand{\mpc}{{\rm Mpc}}$
$\newcommand{\kpc}{{\rm kpc}}$
$\newcommand{\kms}{{\rm km   s}^{-1}}$
$\newcommand{\millen}{MS-I}$
$\newcommand{\msii}{MS-II}$
$\newcommand{\lcdm}{\LambdaCDM}$
$\newcommand{\diso}{d_{\rm iso}}$
$\newcommand{\niso}{n_{\rm iso}}$
$\newcommand{\lt}{<}$
$\newcommand{\gt}{>}$
$\newcommand{\z}{{\it z}}$
$\newcommand{\gmag}{{\it g}}$
$\newcommand{\rmag}{{\it r}}$
$\newcommand{\imag}{{\it i}}$
$\newcommand$
$\newcommand{\oii}{\hbox{\sc[O ii]}}$
$\newcommand{\DMpeak}{\vec{x}_{\delta_{\mathrm{DM,peak}}}^{\mathrm{Baseline}}}$
$\newcommand{\Mstarpeak}{\vec{x}_{\delta_{\mathrm{\star,peak}}}^{\mathrm{Baseline}}}$
$\newcommand{\Galpeak}{\vec{x}_{\delta_{\mathrm{gal,peak}}}^{\mathrm{Baseline}}}$</div>



<div id="title">

# Quantifying the Impact of Incompleteness on Identifying and Interpreting Galaxy Protocluster Populations with the TNG-Cluster Simulation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.03836-b31b1b.svg)](https://arxiv.org/abs/2504.03836)<mark>Appeared on: 2025-04-08</mark> -  _Key Figures: 3, 7, 9, and 13. The manuscript contains 28 pages and 14 figures. Submitted to ApJ. Comments are welcome_

</div>
<div id="authors">

Devontae~C.~Baxter, et al.

</div>
<div id="abstract">

**Abstract:** We use the TNG-Cluster simulation to examine how stellar mass and star formation rate (SFR) incompleteness affect the identification of density peaks within galaxy protoclusters at different redshifts. We focus on 352 protoclusters, defined as the set of galaxies that will reside within the virialized region of a $z=0$ cluster halo with total mass $\sim10^{14.3-15.5} \msun$ , and consider only galaxies with $\mstar > 10^{8.5} \msun$ (our baseline) at any redshift. We find that $\mstar$ -limited ( $\mstar > 10^{9.5} \msun$ ) and SFR-limited ( $\mathrm{SFR} > 10 \msun \mathrm{yr}^{-1}$ ) subpopulations only recover the true highest galaxy density peak in $\lesssim40\%$ of cases within an accuracy of $1.0$ pMpc ( $\sim2-2.5\arcmin$ ) at $z > 2$ . We find that at $z>2$ the highest galaxy density peaks do not generally coincide with the highest dark matter or stellar mass density peaks, and this separation is larger than 0.5 pMpc in $\sim80\%$ of cases. Consequently, the region surrounding the true highest galaxy density peaks are not generally sites of enhanced star formation or accelerated mass growth relative to the remainder of the protocluster. Lastly, we find that the $\sim4\arcmin$ apertures typically used to define spectroscopically-confirmed protoclusters are generally much smaller than the $8\arcmin$ apertures needed to study the progenitors of the most massive galaxy clusters at $z > 2$ .

</div>

<div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.03836"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

431  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

14  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
